In [131]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
import keras.backend as K
from keras.layers import BatchNormalization
import h5py

In [30]:
def unpickle(filename):  #解pickle数据
    import pickle
    with open(filename, 'rb') as f:
        dataSet = pickle.load(f, encoding = 'bytes')
    return dataSet

def splitData(dataSet):
    trainSet = dataSet[b'data']
    trainLabel = dataSet[b'labels']
    return trainSet, np.mat(trainLabel).T

def visualSomeImage(trainSet, trainLabel, labelNames):  #可视化部分图形
    trainLabel = np.array(trainLabel)
    numClass = len(labelNames)
    sample_per_class = 7   #在一张图片里可视化7x10
    for y, cls in enumerate(labelNames):
        idxs = np.flatnonzero(trainLabel == y)
        idxs = np.random.choice(idxs, sample_per_class, replace = False)
        for i, idx in enumerate(idxs):
            plt_idx = i * numClass + y + 1
            plt.subplot(sample_per_class, numClass, plt_idx)
            image = trainSet[idx].reshape(3, 32, 32).transpose(1, 2, 0).astype('float')
            plt.imshow(image.astype('uint8'))
            plt.axis('off')
            if i == 0:
                plt.title(cls)
    plt.show()
                

def get_All_Data():  #加载图片数据集，并且划分为训练集和测试集（首先并没有划分dev集）
    trainFile1 = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\data_batch_1'  #其中之一的训练集
    trainFile2 = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\data_batch_2'  #其中之一的训练集
    trainFile3 = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\data_batch_3'  #其中之一的训练集
    trainFile4 = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\data_batch_4'  #其中之一的训练集
    trainFile5 = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\data_batch_5'  #其中之一的训练集
    labelFile = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\batches.meta'  #返回的的是下标的英文名字
    testFile = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\test_batch'  #测试集合

    dataSet = unpickle(trainFile1)  #首先使用的是一个数据集，看看效果
    trainSet1, trainLabel1 = splitData(dataSet)   #训练集
    dataSet = unpickle(trainFile2)  
    trainSet2, trainLabel2 = splitData(dataSet)   #训练集
    dataSet = unpickle(trainFile3)  
    trainSet3, trainLabel3 = splitData(dataSet)   #训练集
    dataSet = unpickle(trainFile4)  
    trainSet4, trainLabel4 = splitData(dataSet)   #训练集
    dataSet = unpickle(trainFile5)  
    trainSet5, trainLabel5 = splitData(dataSet)   #训练集

    trainSet = np.concatenate((trainSet1, trainSet2), axis = 0)
    trainLabel = np.concatenate((trainLabel1, trainLabel2), axis = 0)
    dataSet = unpickle(testFile)
    testSet, testLabel = splitData(dataSet)  #测试集
    
    #随机选取1000张图片作测试，看准确度
    '''
    randIndex = np.random.permutation(10000)
    shuffled_X = testAllSet[randIndex, :]
    shuffled_Y = testAllLabel[randIndex, :]
    testSet = shuffled_X[:1000, :]   #随机取1000个数据集作为测试
    testLabel = shuffled_Y[:1000, :]
    '''
    label = unpickle(labelFile)  
    labelNames = label[b'label_names'] #label的名字,例如'cat'
    #visualSomeImage(trainSet, trainLabel, labelNames)  #可视化

    #normlization
    trainSet, testSet = trainSet.astype(float), testSet.astype(float)
    mean_image = np.mean(trainSet, axis = 0)
    trainSet -= np.mat(mean_image)
    testSet -= mean_image

    return trainSet1, trainLabel1, testSet, testLabel, labelNames
    
def miniBatch(trainSet, trainLabel,  batchSize = 256):  #将训练数据划分为minibatch，然后进行训练
    m = trainSet.shape[0] #训练集的总数
    mini_batches = []  #划分训练集存储的地方

    #先打乱顺序，然后再mini-batch
    randIndex = np.random.permutation(m)
    shuffled_X = trainSet[randIndex, :]
    shuffled_Y = trainLabel[randIndex, :]

    num_complete_minibatches = int(m/batchSize)
    for k in range(num_complete_minibatches):
        minibatch_X = shuffled_X[k*batchSize : (k+1)*batchSize, :]
        minibatch_Y = shuffled_Y[k*batchSize : (k+1)*batchSize, :]
        minibatch = (minibatch_X, minibatch_Y)
        mini_batches.append(minibatch)

    if m % batchSize != 0:  #如果数据不是minibatch的整数倍，对剩下的单独处理
        minibatch_X = shuffled_X[num_complete_minibatches*batchSize:, :]
        minibatch_Y = shuffled_Y[num_complete_minibatches*batchSize:, :]
        mini_batches.append((minibatch_X, minibatch_Y))

    return mini_batches

def convert_to_one_hot(Y, C):  #将标签变为数量乘以10（这里有10类）
    Y = np.eye(C)[Y].T
    return Y

In [31]:
def load_dataset():  #吴恩达老师的手势数据集
    train_dataset = h5py.File(r'E:\jupyter notebook\finger_sign_dataSet\train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File(r'E:\jupyter notebook\finger_sign_dataSet\test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


In [99]:
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = load_dataset()
trainSet, testSet = train_set_x_orig/255, test_set_x_orig/255
train_set_y_orig = np.array(list(map(int, train_set_y_orig.reshape(-1, 1))))
test_set_y_orig = np.array(list(map(int, test_set_y_orig.reshape(-1, 1))))
trainLabel = convert_to_one_hot(train_set_y_orig, 6).T
testLabel = convert_to_one_hot(test_set_y_orig, 6).T
m = trainLabel.shape[0]

In [82]:

trainFile1 = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\data_batch_1'  #其中之一的训练集
labelFile = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\batches.meta'  #返回的的是下标的英文名字

dataSet = unpickle(trainFile1)  #首先使用的是一个数据集，看看效果
trainSet, trainLabel = splitData(dataSet)   #训练集
label = unpickle(labelFile)  
labelNames = label[b'label_names'] #label的名字,例如'cat'

testFile = r'F:\数据集\斯坦福大学数据集\cifar-10-python\cifar-10-batches-py\test_batch'  #测试集合
dataSet = unpickle(testFile)  #首先使用的是一个数据集，看看效果
testSet, testLabel = splitData(dataSet)   #训练集
m = trainSet.shape[0]

trainSet = trainSet.reshape(m, 3, 32, 32).transpose(0, 2, 3, 1)/255  #将数据变为图片形式，然后输入训练,记住需要归一化
trainLabel = np.array(list(map(int, trainLabel)))
trainLabel = convert_to_one_hot(trainLabel, 10).T

testSet = testSet.reshape(m, 3, 32, 32).transpose(0, 2, 3, 1)/255  #将数据变为图片形式，然后输入训练,记住需要归一化
testLabel = np.array(list(map(int, testLabel)))
testLabel = convert_to_one_hot(testLabel, 10).T

In [137]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(16, (3, 3), padding='same', input_shape=(64, 64, 3)))
#model.add(BatchNormalization(axis = 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.8))

model.add(Conv2D(32, (3, 3), padding='valid'))
#model.add(BatchNormalization(axis = 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.8))

model.add(Conv2D(64, (3, 3), padding='valid'))
#model.add(BatchNormalization(axis = 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.8))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

opti = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=opti, metrics=['accuracy'])



In [150]:
model.fit(trainSet, trainLabel, batch_size=64, epochs=4)

Epoch 1/4
1080/1080 [==============================] - 4s - loss: 0.2122 - acc: 0.9250     
Epoch 2/4
1080/1080 [==============================] - 4s - loss: 0.1657 - acc: 0.9463     
Epoch 3/4
1080/1080 [==============================] - 4s - loss: 0.1786 - acc: 0.9426     
Epoch 4/4
1080/1080 [==============================] - 4s - loss: 0.1414 - acc: 0.9593     


In [153]:
score = model.evaluate(testSet, testLabel, batch_size=64)

120/120 [==============================] - 0s     


In [154]:
score

[0.18709770838419595, 0.94166667858759567]

In [158]:
from keras.models import load_model

model.save(r'E:\jupyter notebook\store_model\storeModel.h5')  # creates a HDF5 file 'my_model.h5'

#model = load_model('my_model.h5')